In [1]:
%reload_ext jetisu.query_idr_magic

# Range

When appropriately constrained in a query, the `range` relation returns
all elements in a range of `N` equally spaced values `FromStart` to
`ToStop`.

The `range` relation is used to illustrate three aspects of
sigma complete relations:

1.  Multi-row result sets

2.  The folding of constraints from the query environment down to create
    a new intensionally defined relation that may be queried without a
    `WHERE` clause

3.  Querying a relation from multiple directions as we normally do for
    data relations.

Here is the definition MiniZinc:

In [2]:
%%jetisu_show
range

```

predicate range(var float: from_start,
                var float: to_stop,
                var int: n,
                var int: nth,
                var float: nth_value) =
let {
  constraint n > 0;
  constraint 0 <= nth /\ nth <= n-1;
  constraint nth_value = from_start + (to_stop-from_start) / (n-1) * nth ;
} in true;
```

The first example query illustrates how an sigma complete
relation may return multiple rows when queried, giving a result which starts at 10, finishes at 20 and has 5 values in
the sequence:

In [3]:
%%jetisu_query
select * from range where N=5 and From_Start=10 and To_Stop = 20;

|from_start|to_stop|n|nth|nth_value|
|----|----|----|----|----|
|10.0|20.0|5.0|0|10|
|10.0|20.0|5.0|1|12.5|
|10.0|20.0|5.0|2|15|
|10.0|20.0|5.0|3|17.5|
|10.0|20.0|5.0|4|20|

The second example query shows how the contents of `WHERE` clauses may
be folded into the predicate of the relation.

```

select * from range where N=5 and FromStart=10 and ToStop = 20 and nthValue < 13;
```



| FromStart | ToStop | N | nth | nthValue |
| :-------- | :----- | :- | :-- | :------- |
| 10.0      | 20.0   | 5 | 0   | 10       |
| 10.0      | 20.0   | 5 | 1   | 12.5     |

Second query results

It is an implementation detail as to what plan was followed to generate
this result. Here are a couple of alternative plans:

  - Compute the result of the first example query and filter out rows
    which do not have `nthValue < 13`

  - Push the "`nthValue < 13`" constraint down into an sigma complete relation `range1` and query it instead of `range`.

Here is an intensional definition of a relation where the query
constraints in the `WHERE` clause are pushed down into the MiniZinc
definition.

In [4]:
%%jetisu_show
range2

```

predicate range2(var float: from_start,
                var float: to_stop,
                var int: n,
                var int: nth,
                var float: nth_value) =
let {
  constraint n > 0;
  constraint 0 <= nth /\ nth <= n-1;
  constraint nth_value = from_start + (to_stop-from_start) / (n-1) * nth ;

} in true;
constraint nth_value < 13.0;
n = 5 ;
from_start = 10;
to_stop = 20;


```

and here is the result of querying this relation

In [5]:
%%jetisu_query
select * from range2;

|from_start|to_stop|n|nth|nth_value|
|----|----|----|----|----|
|10.0|20.0|5|0|10|
|10.0|20.0|5|1|12.5|

The third example query illustrates how the `range` relation may be
queried from multiple directions, as is usually expected for relations. The following query gives a relation which finds the FromStart and Nth values which belong
to sequences with nthValue of 15.0 which finish at 20 and have 5 values
in each sequence. There are four tuples in the result:

In [6]:
%%jetisu_query
select * from range where N=5 and To_Stop = 20 and nth_Value=15.0;

|from_start|to_stop|n|nth|nth_value|
|----|----|----|----|----|
|15|20.0|5.0|0|15.0|
|13.3333333333|20.0|5.0|1|15.0|
|10|20.0|5.0|2|15.0|
|0|20.0|5.0|3|15.0|

## Next step
You can edit and re-run this example workbook by following the instructions here:
[How to run the example notebooks](https://github.com/DavidPratten/jetisu/blob/main/docs/run_notebooks.md)